In [1]:
library(Seurat)
library(Signac)
library(genomation)
library(GenomicRanges)
library(BSgenome.Hsapiens.UCSC.hg38)
library(Matrix)
library(parallel)
library(foreach)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDupl

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/cellranger_arc/refdata-cellranger-arc-GRCh38-2020-A-2.0.0/genes/genes.gtf.gz"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/WTC11_cluster5/WTC11_cluster5/Kendall/Pairs.tsv.gz"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/WTC11_cluster5/WTC11_cluster5/Kendall/atac_matrix.rds"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/WTC11_HPC_lenient/cluster_5/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/WTC11/1.Genome_wide_prediction/Signac/Signac.250325/"
celltype = "WTC11_cluster5"

Import candidate E-G pairs

In [3]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ATAC matrix

In [4]:
matrix.atac = readRDS(path.matrix.atac_count)

Import RNA matrix

In [5]:
matrix.rna_count = read.csv(path.matrix.rna_count,
                            row.names = 1,
                            check.names = F)
matrix.rna_count = Matrix(as.matrix(matrix.rna_count), sparse = TRUE)
matrix.rna_count = matrix.rna_count[,colnames(matrix.atac)]
matrix.rna = NormalizeData(matrix.rna_count)
rm(matrix.rna_count)

Map gene names

In [6]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
    library(dplyr)
    library(data.table)
    
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [7]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:Biostrings’:

    collapse, intersect, setdiff, setequal, union


The following object is masked from ‘package:XVector’:

    slice


The following objects are masked from ‘package:GenomicRanges’:

    intersect, setdiff, union


The following object is masked from ‘package:GenomeInfoDb’:

    intersect


The following objects are masked from ‘package:IRanges’:

    collapse, desc, intersect, setdiff, slice, union


The following objects are masked from ‘package:S4Vectors’:

    first, intersect, rename, setdiff, setequal, union


The following objects are masked from ‘package:BiocGenerics’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, 

Prepare background data

In [8]:
bed.peak = pairs.E2G
mcols(bed.peak) = NULL
bed.peak = unique(bed.peak)
bed.peak = bed.peak[seqnames(bed.peak) %in% paste("chr",c(1:22,"X","Y"), sep = "")]
bed.peak

GRanges object with 152404 ranges and 0 metadata columns:
           seqnames              ranges strand
              <Rle>           <IRanges>  <Rle>
       [1]     chr1 100034573-100034728      *
       [2]     chr1 100037552-100039045      *
       [3]     chr1 100049383-100049815      *
       [4]     chr1 100132371-100133554      *
       [5]     chr1     1001396-1002114      *
       ...      ...                 ...    ...
  [152400]     chrX       997278-998378      *
  [152401]     chrX   99736932-99737301      *
  [152402]     chrX   99777683-99778218      *
  [152403]     chrX     9981419-9981826      *
  [152404]     chrX     9995697-9996195      *
  -------
  seqinfo: 23 sequences from an unspecified genome; no seqlengths

In [9]:
meta.features = RegionStats(bed.peak,
                            genome = BSgenome.Hsapiens.UCSC.hg38)
rownames(meta.features) = paste(bed.peak@seqnames,bed.peak@ranges,sep = "-")
meta.features$seqnames = as.character(bed.peak@seqnames)
meta.features = cbind(meta.features,
                      FindTopFeatures(matrix.atac)[rownames(meta.features),])
meta.features

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,⋯,GT,TA,TC,TG,TT,GC.percent,sequence.length,seqnames,count,percentile
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>
chr1-100034573-100034728,13,4,14,12,8,2,0,10,10,5,⋯,13,12,9,15,16,39.10256,156,chr1,42,0.38838876
chr1-100037552-100039045,75,64,90,43,72,156,130,107,80,148,⋯,75,45,96,84,69,62.11513,1494,chr1,674,0.96430540
chr1-100049383-100049815,19,23,25,29,27,38,5,44,22,23,⋯,28,28,30,43,38,45.49654,433,chr1,35,0.27445474
chr1-100132371-100133554,62,56,77,43,52,105,62,93,81,80,⋯,64,43,71,85,97,54.81419,1184,chr1,608,0.95542768
chr1-1001396-1002114,18,16,45,6,21,112,71,51,43,89,⋯,25,3,38,42,19,73.99166,719,chr1,214,0.86378310
chr1-100147835-100148361,63,23,40,47,34,17,5,20,49,24,⋯,26,26,12,55,48,40.41746,527,chr1,47,0.44944358
chr1-100199663-100200848,70,62,117,41,105,101,19,98,87,90,⋯,45,29,70,85,49,55.81788,1186,chr1,138,0.80288575
chr1-10021-10505,74,75,1,0,8,168,6,69,1,5,⋯,2,67,3,1,0,54.22680,485,chr1,27,0.08161203
chr1-100233547-100234004,56,30,43,25,36,18,5,27,31,24,⋯,20,31,14,28,42,41.26638,458,chr1,54,0.51637752


Run Signac prediction

In [10]:
Sys.setenv(OPENBLAS_NUM_THREADS=1)
PredictionSignac = function(pairs.E2G,
                            data.RNA,
                            data.ATAC,
                            meta.features,
                            n_sample = 200,
                            cor_method = qlcMatrix::corSparse,
                            n.core = 16){
  
  
  my.cluster <- parallel::makeCluster(
    n.core,
    type = "PSOCK"
  )
  doParallel::registerDoParallel(cl = my.cluster)
  
  pairs.E2G =  
    foreach (gene.tmp = unique(pairs.E2G$TargetGene),
             .combine = 'c',
             .packages = c("Signac",
                           "Matrix")) %dopar% {
                             
                             pairs.E2G.tmp = pairs.E2G[pairs.E2G$TargetGene == gene.tmp]
                             
                             data.RNA.tmp.t = t(data.RNA[gene.tmp, 
                                                         , drop = FALSE])
                             data.ATAC.tmp.t = t(data.ATAC[pairs.E2G.tmp$PeakName, 
                                                           , drop = FALSE])
                             
                             coef.result <- cor_method(X = data.ATAC.tmp.t, 
                                                       Y = data.RNA.tmp.t)
                             
                             
                             meta.features.bg <- meta.features[!meta.features$seqnames %in% as.character(seqnames(pairs.E2G.tmp)), ]
                             meta.features.peaks <- meta.features[pairs.E2G.tmp$PeakName, ]
                             bg.peaks <- lapply(X = seq_len(length.out = nrow(x = meta.features.peaks)), 
                                                FUN = function(x) {
                                                  MatchRegionStats(meta.feature = meta.features.bg, 
                                                                   query.feature = meta.features.peaks[x, , drop = FALSE], 
                                                                   features.match = c("GC.percent", "count", "sequence.length"), 
                                                                   n = n_sample, 
                                                                   verbose = FALSE)
                                                })
                             data.ATAC.bg.t <- t(data.ATAC[unlist(x = bg.peaks),, 
                                                           drop = FALSE])
                             bg.coef <- cor_method(X = data.ATAC.bg.t, Y = data.RNA.tmp.t)
                             rownames(bg.coef) <- colnames(data.ATAC.bg.t)
                             for (j in seq(length(pairs.E2G.tmp))) {
                               coef.use <- bg.coef[(((j - 1) * n_sample) + 
                                                      1):(j * n_sample), ]
                               z <- (coef.result[j] - mean(x = coef.use))/sd(x = coef.use)
                               pairs.E2G.tmp[j]$zscore <- z
                             }
                             pairs.E2G.tmp$pvalue = 1 - pnorm(pairs.E2G.tmp$zscore)
                             pairs.E2G.tmp
                           }
  
  parallel::stopCluster(cl = my.cluster)
  
  return(pairs.E2G)
}

In [11]:
pairs.E2G.filter = pairs.E2G[pairs.E2G$TargetGene %in% rownames(matrix.rna.rename) &
                             pairs.E2G$PeakName %in% rownames(matrix.atac)]
mcols(pairs.E2G.filter)[,c("zscore","pvalue")] = NA
pairs.E2G.filter

GRanges object with 9161951 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034573-100034728      * |         AGL
        [2]     chr1 100034573-100034728      * |       ALG14
        [3]     chr1 100034573-100034728      * |      CDC14A
        [4]     chr1 100034573-100034728      * |     COL11A1
        [5]     chr1 100034573-100034728      * |         DBT
        ...      ...                 ...    ... .         ...
  [9161947]     chrX     9995697-9996195      * |       TBL1X
  [9161948]     chrX     9995697-9996195      * |      TCEANC
  [9161949]     chrX     9995697-9996195      * |      TMSB4X
  [9161950]     chrX     9995697-9996195      * |     TRAPPC2
  [9161951]     chrX     9995697-9996195      * |        WWC3
                          PeakName               PairName    zscore    pvalue
                       <character>            <character>

In [12]:
start_time <- Sys.time()
pairs.E2G.res = PredictionSignac(pairs.E2G.filter,
                                 matrix.rna.rename,
                                 matrix.atac,
                                 meta.features,
                                 8)
end_time <- Sys.time()
execution_time <- end_time - start_time
execution_time

Time difference of 16.99963 hours

In [13]:
execution_time

Time difference of 16.99963 hours

Save results

In [14]:
dir.create(dir.output,recursive = T)
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 9161951 ranges and 5 metadata columns:
            seqnames              ranges strand |  TargetGene
               <Rle>           <IRanges>  <Rle> | <character>
        [1]     chr1 100034573-100034728      * |         AGL
        [2]     chr1 100037552-100039045      * |         AGL
        [3]     chr1 100049383-100049815      * |         AGL
        [4]     chr1 100132371-100133554      * |         AGL
        [5]     chr1 100147835-100148361      * |         AGL
        ...      ...                 ...    ... .         ...
  [9161947]     chrX   98573465-98574181      * |      NAP1L3
  [9161948]     chrX   98582054-98582692      * |      NAP1L3
  [9161949]     chrX   98605977-98606332      * |      NAP1L3
  [9161950]     chrX   98607780-98608181      * |      NAP1L3
  [9161951]     chrX   98652221-98652822      * |      NAP1L3
                          PeakName               PairName    zscore    pvalue
                       <character>            <character>

In [15]:
df.output = as.data.frame(pairs.E2G.res)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,width,strand,TargetGene,PeakName,PairName,zscore,pvalue,CellType
<fct>,<int>,<int>,<int>,<fct>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
chr1,100034573,100034728,156,*,AGL,chr1-100034573-100034728,chr1-100034573-100034728_AGL,-0.18502678,0.57339599,WTC11_cluster5
chr1,100037552,100039045,1494,*,AGL,chr1-100037552-100039045,chr1-100037552-100039045_AGL,2.31601087,0.01027884,WTC11_cluster5
chr1,100049383,100049815,433,*,AGL,chr1-100049383-100049815,chr1-100049383-100049815_AGL,1.33611427,0.09075596,WTC11_cluster5
chr1,100132371,100133554,1184,*,AGL,chr1-100132371-100133554,chr1-100132371-100133554_AGL,-0.46853281,0.68029819,WTC11_cluster5
chr1,100147835,100148361,527,*,AGL,chr1-100147835-100148361,chr1-100147835-100148361_AGL,-3.79010726,0.99992471,WTC11_cluster5
chr1,100199663,100200848,1186,*,AGL,chr1-100199663-100200848,chr1-100199663-100200848_AGL,1.27918616,0.10041575,WTC11_cluster5
chr1,100233547,100234004,458,*,AGL,chr1-100233547-100234004,chr1-100233547-100234004_AGL,-0.32691413,0.62813358,WTC11_cluster5
chr1,100249099,100250262,1164,*,AGL,chr1-100249099-100250262,chr1-100249099-100250262_AGL,0.62034549,0.26751517,WTC11_cluster5
chr1,100265655,100266996,1342,*,AGL,chr1-100265655-100266996,chr1-100265655-100266996_AGL,-0.45765524,0.67639992,WTC11_cluster5


In [16]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] data.table_1.15.2                 dplyr_1.1.4                      
 [3] foreach_1.5.2                     Matrix_1.6